In [1]:
import os
import re
import random
import string
import requests

import numpy as np
import pandas as pd
import geopandas as gpd

import arcpy
import arcpy.sa as sa
import arcpy.mp as mp
arcpy.CheckOutExtension("Spatial")

import time

#### Define a function to clean the output folder of all files. But it can't ignore locks, eg if a file is open in Excel:

In [2]:
def clean_folder(output_folder_path):
    for filename in os.listdir(output_folder_path):
        file_path = os.path.join(output_folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                import shutil
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Could not delete {file_path}: {e}")

#### Define a function to run zonal statistics. The variables are:  
##### zone_fc - feature class to act as the aggregator, for us that's trimmed CBG polygons  
##### zone_field - the field that acts as a unique identifier, for us that will be GEOIDFQ  
##### raster_path - the path to the raster file used for the analysis
##### output_folder - the path to the output folder for the DBF file containing statistics
##### stat_type - the type of statistics, MEAN, MAXIMUM, MINIMUM, etc  
#####
#### Coordinate system must match between polygon and rasters being analyzed!!

In [3]:
def func_zonal_stats(zone_fc, zone_field, raster_path, output_folder, stat_type):
    import os
    import arcpy
    import arcpy.sa as sa
    arcpy.CheckOutExtension("Spatial")
    arcpy.env.overwriteOutput = True

    # Sanitize raster base name for use in DBF field/table names (max 8 chars is safest for DBF)
    raster_base = os.path.splitext(os.path.basename(raster_path))[0]
    raster_base_clean = raster_base.replace(" ", "_")[:32]  # Max safe length for filename
    stat_type_clean = stat_type.lower()

    out_table_name = f"{raster_base_clean}_{stat_type_clean}.dbf"

    out_table_path = os.path.join(output_folder, "temp", out_table_name)
    
    out_temp_folder = os.path.join(output_folder, "temp")
    
    #Create OUTPUT folder which was pasased in as variable, and nested TEMP folder if they don't exist already
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(out_temp_folder, exist_ok=True)
    

    # Run zonal statistics
    sa.ZonalStatisticsAsTable(
        in_zone_data=zone_fc,
        zone_field=zone_field,
        in_value_raster=raster_path,
        out_table=out_table_path,
        statistics_type=stat_type,
        ignore_nodata="DATA"
    )
    
    return out_table_path  # return path to the table so you can print for confirmation

#### Function to calculate whether the zone_fc vector layer has any contact with the vector_path layer

In [4]:
def func_vector_match(zone_fc, zone_field, vector_path, output_folder, match_type):
    import os
    import arcpy
    arcpy.env.overwriteOutput = True

    # Sanitize base name
    vector_base = os.path.splitext(os.path.basename(vector_path))[0]
    vector_base_clean = vector_base.replace(" ", "_")[:32]
    match_type_clean = match_type.lower()

    # Output table name
    out_table_name = f"{vector_base_clean}_{match_type_clean}.dbf"
    out_table_path = os.path.join(output_folder, "temp", out_table_name)

    out_temp_folder = os.path.join(output_folder, "temp")
    
    #Create OUTPUT folder which was pasased in as variable, and nested TEMP folder if they don't exist already. Clean them if they do
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(out_temp_folder, exist_ok=True)

    # Create feature layers
    zone_layer = "zone_layer"
    vector_layer = "vector_layer"
    arcpy.MakeFeatureLayer_management(zone_fc, zone_layer)
    arcpy.MakeFeatureLayer_management(vector_path, vector_layer)

    # New code to dissolve
    dissolved_vector = os.path.join(out_temp_folder, "dissolved_temp.shp")
    # Delete existing output to avoid overwrite issues
    if arcpy.Exists(dissolved_vector):
        arcpy.management.Delete(dissolved_vector)
    arcpy.management.Dissolve(vector_path, dissolved_vector)
    
    # Perform spatial join
    temp_join = "in_memory/temp_join"
    arcpy.analysis.SpatialJoin(
        target_features=zone_layer,
        join_features=dissolved_vector,
        out_feature_class=temp_join,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        match_option=match_type.upper()  # INTERSECT, WITHIN, CONTAINS, etc.
    )

    # Add a binary 1/0 field indicating presence/absence of match
    fieldname_short = f"{match_type_clean}"[:10]  # Ensure DBF-safe
    arcpy.management.AddField(temp_join, fieldname_short, "SHORT")

    # Determine if match occurred by checking for nulls in join fields
    arcpy.management.CalculateField(
        in_table=temp_join,
        field=fieldname_short,
        expression="0 if !Join_Count! is None or !Join_Count! == 0 else 1",
        expression_type="PYTHON3"
    )

    # Export table
    arcpy.conversion.TableToTable(temp_join, out_temp_folder, out_table_name)

    return out_table_path

#### Function to calculate % overlap between zone_fc vector layer has any contact with the vector_path layer

In [5]:
def func_vector_pct(zone_fc, zone_field, vector_path, output_folder):
    import os
    import arcpy

    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("Spatial")

    # Clean name for output (based on intersecting feature class)
    vector_base = os.path.splitext(os.path.basename(vector_path))[0]
    out_name = f"{vector_base[:32]}_pct.shp"
    out_path = os.path.join(output_folder, "temp", out_name)

    out_temp_folder = os.path.join(output_folder, "temp")
    
    #Create OUTPUT folder which was pasased in as variable, and nested TEMP folder if they don't exist already. Clean them if they do
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(out_temp_folder, exist_ok=True)

    # New code to dissolve
    dissolved_vector = os.path.join(out_temp_folder, "dissolved_temp.shp")
    # Delete existing output to avoid overwrite issues
    if arcpy.Exists(dissolved_vector):
        arcpy.management.Delete(dissolved_vector)
    arcpy.management.Dissolve(vector_path, dissolved_vector)
    
    # Step 1: Intersect zone with input vector (attributes from zone_fc retained)
    arcpy.analysis.Intersect([zone_fc, dissolved_vector], out_path, "ALL")

    # Step 2: Add overlap area (short field name)
    arcpy.management.AddField(out_path, "ov_area", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(out_path, [["ov_area", "AREA_GEODESIC"]])

    # Step 3: Get zone area from original features (via temporary copy)
    #zone_copy = os.path.join(output_folder, "zone_temp.shp")
    #arcpy.conversion.FeatureClassToFeatureClass(zone_fc, output_folder, out_name)
    zone_copy_name = "zone_temp.shp"
    zone_copy = os.path.join(out_temp_folder, zone_copy_name)
    arcpy.conversion.FeatureClassToFeatureClass(zone_fc, out_temp_folder, zone_copy_name)

    arcpy.management.AddField(zone_copy, "zn_area", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(zone_copy, [["zn_area", "AREA_GEODESIC"]])

    # Step 4: Join zone area to intersected features
    arcpy.management.JoinField(out_path, zone_field, zone_copy, zone_field, ["zn_area"])

    # Step 5: Add and calculate percent field
    arcpy.management.AddField(out_path, "pct", "FLOAT")
    arcpy.management.CalculateField(
        out_path, "pct",
        expression="(!ov_area! / !zn_area!) * 100 if !zn_area! else 0",
        expression_type="PYTHON3"
    )

    #Step 6: Delete the temporary zone shapefile
    arcpy.management.Delete(zone_copy)
    
    return out_path

#### Function to calculate ACREAGE overlap between zone_fc vector layer has any contact with the vector_path layer

In [6]:
def func_vector_acres(zone_fc, zone_field, vector_path, output_folder):
    import os
    import arcpy

    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("Spatial")

    # Clean name for output shapefile
    vector_base = os.path.splitext(os.path.basename(vector_path))[0]
    out_name = f"{vector_base[:32]}_acres.shp"
    out_path = os.path.join(output_folder, "temp", out_name)

    out_temp_folder = os.path.join(output_folder, "temp")
    
    #Create OUTPUT folder which was pasased in as variable, and nested TEMP folder if they don't exist already
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(out_temp_folder, exist_ok=True)


    # Delete existing output to avoid overwrite issues
    if arcpy.Exists(out_path):
        arcpy.management.Delete(out_path)

    # New code to dissolve
    dissolved_vector = os.path.join(out_temp_folder, "dissolved_temp.shp")
    # Delete existing output to avoid overwrite issues
    if arcpy.Exists(dissolved_vector):
        arcpy.management.Delete(dissolved_vector)
    arcpy.management.Dissolve(vector_path, dissolved_vector)
    
    # Intersect zone features with vector_path (attributes from zone_fc retained)
    arcpy.analysis.Intersect([zone_fc, dissolved_vector], out_path, "ALL")

    # Add area field (in acres)
    arcpy.management.AddField(out_path, "acres", "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(out_path, [["acres", "AREA_GEODESIC"]], area_unit="ACRES")

    return out_path

In [7]:
def func_spatialcount(zone_fc, zone_field, vector_path, output_folder, radius):
    import os
    import arcpy

    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("Spatial")

    # Clean name for output shapefile
    vector_base = os.path.splitext(os.path.basename(vector_path))[0]
    out_name = f"{vector_base[:32]}_count.shp"
    out_path = os.path.join(output_folder, "temp", out_name)

    out_temp_folder = os.path.join(output_folder, "temp")
    
    #Create OUTPUT folder which was pasased in as variable, and nested TEMP folder if they don't exist already. Clean them if they do
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(out_temp_folder, exist_ok=True)

    # Delete existing output to avoid overwrite issues
    if arcpy.Exists(out_path):
        arcpy.management.Delete(out_path)

    # Step 1: Perform spatial join to count features within radius
    spatial_join_temp = os.path.join(output_folder, "temp_spjoin.shp")
    arcpy.analysis.SpatialJoin(
        target_features=zone_fc,
        join_features=vector_path,
        out_feature_class=spatial_join_temp,
        join_operation="JOIN_ONE_TO_MANY",
        join_type="KEEP_ALL",
        match_option="WITHIN_A_DISTANCE",
        search_radius=radius,
        distance_field_name=None
    )

    # Step 2: Dissolve to get count of joined features per zone
    arcpy.analysis.Statistics(
        in_table=spatial_join_temp,
        out_table=out_path,
        statistics_fields=[["JOIN_FID", "COUNT"]],
        case_field=zone_field
    )

    # Step 3 (optional): Delete temp join file
    arcpy.management.Delete(spatial_join_temp)

    return out_path

#### Define a function to combine the multiple DBF files into one CSV. The field name in the CSV is taken from the DBF name, which originally was taken from the raster:

In [8]:
def func_combine_tables(in_folder, out_folder, key_field, output_csv):
    #key_field = "GEOIDFQ"
    #output_csv = "combined_zonal_stats.csv"
    #create list of DBF files in the in_folder
    dbf_files = [f for f in os.listdir(in_folder) if f.lower().endswith(".dbf")]
    merged_df = None

    for dbf in dbf_files:
        dbf_path = os.path.join(in_folder, dbf)
        
        # Get all non-geometry, non-OID fields
        fields = [f.name for f in arcpy.ListFields(dbf_path) if f.type not in ("Geometry", "OID")]

        if key_field not in fields or len(fields) < 2:
            continue  # Skip if missing key or only one useful field

        stat_field = fields[-1]  # Use rightmost field
        table = arcpy.da.TableToNumPyArray(dbf_path, [key_field, stat_field])
        df = pd.DataFrame(table)

        # Rename stat field to the DBF base name
        stat_name = os.path.splitext(dbf)[0]
        df = df.rename(columns={stat_field: stat_name})

        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=key_field, how="outer")

    # Save combined CSV
    if merged_df is not None:
        output_csv_path = os.path.join(out_folder, output_csv)
        merged_df.to_csv(output_csv_path, index=False)
        return output_csv_path
    else:
        raise ValueError("No valid DBFs found or no data to merge.")

    return output_csv_path

#### Define a function to join the CSV back onto the CBG shapefile as a new output:

In [9]:
def func_csv_join_to_shp(shapefile_cbg_path, merged_csv):
    import os
    import arcpy

    arcpy.env.overwriteOutput = True

    # Output path based on CSV folder
    output_folder = os.path.dirname(merged_csv)
    output_shp = os.path.join(output_folder, "combined_zonal_shapes.shp")

    # Assume GEOIDFQ is the common key
    arcpy.management.MakeFeatureLayer(shapefile_cbg_path, "cbg_layer")
    arcpy.management.MakeTableView(merged_csv, "csv_table")

    arcpy.management.AddJoin("cbg_layer", "GEOIDFQ", "csv_table", "GEOIDFQ", "KEEP_COMMON")

    arcpy.management.CopyFeatures("cbg_layer", output_shp)

    # Clean up layers
    arcpy.management.Delete("cbg_layer")
    arcpy.management.Delete("csv_table")

    return output_shp

#### Get the path of this notebook and use it to generate path for input shapefile:

In [10]:
current_dir = os.getcwd()
shapefile_cbg = r"input\cbg_kontur.shp"
shapefile_cbg_path = os.path.join(current_dir, shapefile_cbg)
output_folder_path = os.path.join(current_dir, "output")


print(f"Location of this notebook: {current_dir}")
print(f"Location of CBG SHP: {shapefile_cbg_path}")
print(f"Location for output: {output_folder_path}")

Location of this notebook: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis
Location of CBG SHP: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_kontur.shp
Location for output: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\output


#### Define output folder and create it if it doesn't exist. Then clean it in case it has files already:

In [11]:
output_folder = os.path.join(current_dir, "output")
temp_folder = os.path.join(current_dir, "output", "temp")
os.makedirs(output_folder, exist_ok=True)
clean_folder(output_folder_path)

#### The following lines run the zonal statistics function for each raster independently.  
#### The outputs are all generated as DBF files with names originating from the rasters.  
#### As many of these lines can be added as necessary, just change the inputs for the appropriate raster and statistic:

In [12]:
func_zonal_stats(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\staterf_inann.tif", output_folder, "MEAN")


'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\staterf_inann_mean.dbf'

In [13]:
func_zonal_stats(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\igtn_prob_test.tif", output_folder, "MAXIMUM")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\igtn_prob_test_maximum.dbf'

In [14]:
func_zonal_stats(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\n10_landslide_susc.tif", output_folder, "MAXIMUM")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\n10_landslide_susc_maximum.dbf'

In [15]:
func_zonal_stats(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\cat4_mom_slosh_hightide.tif", output_folder, "MAXIMUM")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\cat4_mom_slosh_hightide_maximum.dbf'

#### The following lines run Spatial Satistics for each vector dataset independently:

In [16]:
func_vector_match(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\slrxa_3pt2ft.shp", output_folder, "INTERSECT")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\slrxa_3pt2ft_intersect.dbf'

In [17]:
func_vector_pct(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\slrxa_3pt2ft.shp", output_folder)

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\slrxa_3pt2ft_pct.shp'

In [18]:
func_vector_acres(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\slrxa_3pt2ft.shp", output_folder)

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\slrxa_3pt2ft_acres.shp'

In [19]:
func_vector_match(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\environmental\FEMA_SFHA.shp", output_folder, "INTERSECT")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\FEMA_SFHA_intersect.dbf'

#### The follow lines run Spatial Joins to count features within a certain distance from each CBG:

In [20]:
func_spatialcount(shapefile_cbg_path, "GEOIDFQ", r"C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\facilities\emergency_shelters.shp", output_folder, "0.5 miles")

'C:\\GITHUB\\CCSVI\\Scripts\\Spatial_Analysis\\output\\temp\\emergency_shelters_count.shp'

#### This runs the function to take the previously generated DBF files and merge them all to one CSV file with fieldnames from the DBFs:

In [21]:
merged_csv = func_combine_tables(temp_folder,output_folder_path, "GEOIDFQ", "combined_zonal_stats.csv")
print(merged_csv)

C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\output\combined_zonal_stats.csv


#### This joins the CSV back onto the shapefile as a new output

In [22]:
#print(func_csv_join_to_shp(shapefile_cbg_path, merged_csv))

In [23]:
#print("Finished at:", time_mod.strftime("%Y-%m-%d %H:%M:%S", time_mod.localtime(time_mod.time())))